# Chapter3 確率的勾配降下法

## 3.1 勾配降下法

### 誤差関数

・回帰問題なら**二乗誤差**<br>

$$
\begin{equation}
E(w) = \frac{1}{2}\sum_{n=1}^{N} \| \mathbf{d_n}-\mathbf{y}(\mathbf{x_n};\mathbf{w})\|^2
\end{equation}
$$

・分類問題なら**クロスエントロピー**<br>

$$
\begin{equation}
E(w) = -\sum_{n=1}^{N} \sum_{k=1}^{K} d_{nk}\log y_k(\mathbf{x_n};\mathbf{w})
\end{equation}
$$

<br>
→学習の目的は誤差関数をネットワークパラメータについて最小化すること<br>

**勾配降下法**:パラメータを初期値から少しづつ変化させて誤差関数の極小値を得る<br>

※誤差関数は凸関数とは限らないので局所解に陥る可能性はある。

### パラメータの更新

勾配降下法では以下のように勾配が大きい方に向かってパラメータを更新する。

$$
\begin{equation}
\mathbf w^{t+1} = \mathbf w^{t} - \epsilon \nabla E
\end{equation}
$$
<br>
$$
\begin{equation}
\nabla E \equiv \frac{\partial E}{\partial \mathbf w} = \Biggl[ \frac{\partial E}{\partial w_1} \cdots \frac{\partial E}{\partial w_M} \Biggr]^{\mathrm{T}}
\end{equation}
$$

$\epsilon$は**学習係数**と呼ばれる重み更新の幅を決めるパラメータ。

In [18]:
'''
code from
https://gist.github.com/matsuken92/f92f8e8e4a1f924e6d99#file-2-dim-steepest-descent_quadrant-py
'''
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import *
from matplotlib import animation as ani

sigma = 1
mu     = 3

# set graph range
x_low  = 2
x_high = 8
y_low  = 2
y_high = 8 

# set field
X = np.linspace(x_low, x_high, 1000)
Y = np.linspace(y_low, y_high, 1000)
X, Y = np.meshgrid(X, Y)

# set parameters
D = 2
mu_x = 5 #np.average(x)
mu_y = 5 #np.average(y)
cov = [[1.5,  1.4],
       [1.4,  3.8]]
cov_inv = np.linalg.inv(cov)
cov_det = np.linalg.det(cov)
c = 1./((2 * np.pi) ** (D/2) * np.sqrt(cov_det))
Z = c * np.exp(-0.5 * ((cov_inv[0,0] * (X-mu_x)**2) + \
                       (2 * cov_inv[0, 1]) * (X-mu_x)*(Y-mu_y) + \
                       (cov_inv[1,1] * (Y-mu_y) ** 2)))

def get_grad_vec(x, y):
    c = 1./((2 * np.pi) ** (D/2) * np.sqrt(cov_det)) 
    Z = c * np.exp(-0.5 * ((cov_inv[0,0] * (x-mu_x)**2) + \
                       (2 * cov_inv[0, 1]) * (x-mu_x)*(y-mu_y) + \
                       (cov_inv[1,1] * (y-mu_y) ** 2)))
    
    grad_x = Z * 2 * ((x -mu_x)*cov_inv[0,0] + (y-mu_y)*cov_inv[0,1])
    grad_y = Z * 2 * ((x -mu_x)*cov_inv[1,0] + (y-mu_y)*cov_inv[1,1])
    return [grad_x, grad_y]

def calc_2val_norm(init_x=3, init_y=3, learning_ratio = 5, precision=5):
    list_xs   = []
    list_ys   = []
    list_nxs  = []
    list_nys  = []
    list_diff = []
    xs = init_x
    ys = init_y
    i = 0
    
    for i in range(500):
        grad_vec = get_grad_vec(xs, ys)
        n_xs = xs - learning_ratio*grad_vec[0]
        n_ys = ys - learning_ratio*grad_vec[1]
        
        list_xs.append(xs)
        list_ys.append(ys)
        list_nxs.append(n_xs)
        list_nys.append(n_ys)

        # judge convergence
        diff = np.sqrt(grad_vec[0]**2 + grad_vec[1]**2)
        list_diff.append(diff)
        if diff < 0.1**precision:
            print("break")
            break
        
        xs = n_xs
        ys = n_ys
        
    ret_dict = {}
    ret_dict['num'] = i+1
    ret_dict['list_xs'] = list_xs
    ret_dict['list_ys'] = list_ys
    ret_dict['list_nxs'] = list_nxs
    ret_dict['list_nys'] = list_nys
    ret_dict['list_diff'] = list_diff
    
    return ret_dict

def animate(i):
    #print "i %d" % i
    list_xs = ret_dict['list_xs']
    list_ys = ret_dict['list_ys']
    list_nxs = ret_dict['list_nxs']
    list_nys = ret_dict['list_nys']
    list_diff = ret_dict['list_diff']
    
    # draw graph
    plt.scatter(list_xs[i], list_ys[i], s=20, c="b", alpha=0.6)
    plt.plot([list_xs[i], list_nxs[i]], [list_ys[i], list_nys[i]])
    plt.title("n %2d, x %.5f, y %.5f, diff %.5f" % (i, list_xs[i], list_ys[i], list_diff[i]))
    
fig = plt.figure(figsize=(7,5))
ret_dict = calc_2val_norm(init_x=5, init_y=8)
interval = np.arange(0.019, 0.1, 0.01)
CS = plt.contour(X, Y, Z, interval)
plt.clabel(CS, inline=1, fontsize=10)
anim = ani.FuncAnimation(fig, animate, frames=ret_dict['num'], blit=True)
anim.save('bi-normdist_decent_anim.mp4', fps=2.5)
    
clip = VideoFileClip("bi-normdist_decent_anim.mp4")
clip.write_gif("bi-normdist_decent_anim.gif")

break


TypeError: 'NoneType' object is not iterable

## 3.2 確率的勾配降下法

### 最急降下法と確率的勾配降下法

誤差関数は全ての訓練データに対する誤差の和

$$
\begin{equation}
E(\mathbf w) = \sum_{n=1}^{N} E_n(\mathbf w)
\end{equation}
$$

**最急降下法**:<br>
全ての訓練データについての誤差関数の勾配に従ってパラメータを更新

$$
\begin{equation}
\mathbf w^{t+1} = \mathbf w^{t} - \epsilon \nabla E
\end{equation}
$$

**確率的勾配降下法**:<br>
一つの訓練データについての誤差関数の勾配に従ってパラメータを更新

$$
\begin{equation}
\mathbf w^{t+1} = \mathbf w^{t} - \epsilon \nabla E_n
\end{equation}
$$

In [ ]:
全てのデータが同じ場合の地形を図示して、データの相関が大きい場合に確率的勾配降下法が有効であることを示す

### 確率的勾配降下法の利点

- 訓練データに冗長性がある場合に計算効率が向上する
- 局所解にトラップされづらくなる
- 学習の途中経過を細かく監視できる
- オンライン学習が可能

## 3.3 ミニバッチの利用

### 最急降下法と確率的勾配降下法の中間

- 少数のサンプル(ミニバッチ)についての誤差関数の勾配に従ってパラメータを更新

- 多クラス分類問題ではミニバッチサイズはクラス数くらい(あまり大きくしない方がよい)

- 全てのクラスをミニバッチに含むようにするのが良い<br>

## 3.4 汎化性能と過適合

### 訓練誤差とテスト誤差

**訓練誤差**: 訓練データに対する誤差　<br>

**テスト誤差**: モデルがまだ見たことがないデータに対する誤差<br>

**過学習**: 訓練誤差は小さいがテスト誤差は大きくなってしまう現象

**早期終了**: テスト誤差が大きくなる前に学習を打ち切ることで過学習を防ぐ

## 3.5 過適合の緩和

### 正則化

正則化: 重みに制限をかける

**L1正則化**:
大部分のパラメータを0にしてスパースな解を得る。



**L2正則化**:
個々のパラメータの大きさを制限して過学習を防止する。

$$
\begin{equation}
E_t(\mathbf{w}) = \frac{1}{N_t}\sum_{n\in \it{D_t}} E_n(\mathbf{w}) + \frac{\lambda}{2}\| \mathbf{w} \|^2
\end{equation}
$$

### ドロップアウト

- 過学習を防止する方法の一つ

- 学習時、ミニバッチを入力するたびに層のユニットを確率1-pで無効化する

- 推論時は、無効化の対象としたユニットの出力を一律にp倍する

## 3.6 学習のトリック

### 3.6.1 データの正規化

### 3.6.2 データ拡張

### 3.6.3 複数ネットの平均

### 3.6.4 学習係数の決め方

### 3.6.5 モメンタム

### 3.6.6 重みの初期化

### 3.6.7 サンプルの順序